In [18]:
import pandas as pd
df = pd.read_parquet('../data/export_df.parquet')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119210 entries, 0 to 119209
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   hotel_id                       119210 non-null  int32         
 1   datum_dolaska                  119210 non-null  datetime64[ns]
 2   datum_kreiranja_rezervacije    119210 non-null  datetime64[ns]
 3   datum_odjave                   119210 non-null  datetime64[ns]
 4   datum_otkazivanja_rezervacije  44199 non-null   object        
 5   broj_odraslih_gostiju          119210 non-null  int64         
 6   broj_djece_gostiju             119210 non-null  float64       
 7   zemlja_gosta                   119210 non-null  object        
 8   kanal_prodaje_id               119210 non-null  int32         
 9   tip_sobe_id                    119210 non-null  int32         
 10  cijena_nocenja                 119210 non-null  float64       
 11  

In [20]:
df['status_rezervacije'].value_counts()

Check-Out    75011
Canceled     42993
No-Show       1206
Name: status_rezervacije, dtype: int64

In [21]:
status_mapping = {'Check-Out': 0, 'Canceled': 1, 'No-Show': 2}

df['status_rezervacije'] = df['status_rezervacije'].map(status_mapping)

Ne mozemo bas da predvidjamo da li ce nam neko odustati, to nam nije zadatak, tako da predvidjamo samo za one koji su check inovali i checkoutovali, tj. zapravo okupirali hotele, pa zato filtriram sledece:

In [22]:
df = df[df['status_rezervacije']==0]

Sad otklanjam nepotrebne kolone

In [23]:
df = df.drop(columns=['datum_otkazivanja_rezervacije','status_rezervacije','datum_kreiranja_rezervacije'], axis = 1)

Kako ne bismo povecavali dimenzije dataseta drasticno (putem dummy variables), mapiramo numericki sve drzave u jednoj koloni, mada je moguce da ovo nije bas najbolja praksa, zbog kategorijske prirode drzava, mapirati ih direktno u skalare mozda nije u potpunosti mudro

In [24]:
df['zemlja_gosta_mapped'] = pd.factorize(df['zemlja_gosta'])[0]
print(df[['zemlja_gosta', 'zemlja_gosta_mapped']])

       zemlja_gosta  zemlja_gosta_mapped
0               PRT                    0
1               PRT                    0
2               GBR                    1
3               GBR                    1
4               GBR                    1
...             ...                  ...
119205          BEL                   13
119206          FRA                    5
119207          DEU                   12
119208          ESP                    3
119209          BEL                   13

[75011 rows x 2 columns]


In [25]:
df = df.drop(columns='zemlja_gosta', axis = 1)                             

In [26]:
df['broj_gostiju'] = df['broj_djece_gostiju'] + df['broj_odraslih_gostiju']

In [27]:
df = df.drop(columns = ['broj_djece_gostiju', 'broj_odraslih_gostiju'], axis = 1)

In [28]:
df = df.reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75011 entries, 0 to 75010
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   index                75011 non-null  int64         
 1   hotel_id             75011 non-null  int32         
 2   datum_dolaska        75011 non-null  datetime64[ns]
 3   datum_odjave         75011 non-null  datetime64[ns]
 4   kanal_prodaje_id     75011 non-null  int32         
 5   tip_sobe_id          75011 non-null  int32         
 6   cijena_nocenja       75011 non-null  float64       
 7   rezervacija_id       75011 non-null  int32         
 8   gost_id              75011 non-null  int32         
 9   zemlja_gosta_mapped  75011 non-null  int64         
 10  broj_gostiju         75011 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int32(5), int64(2)
memory usage: 4.9 MB


Razdvajamo u dva hotela jer cenim da ce oni imati razlicite sobe barem

In [29]:
df1 = df[df['hotel_id']==1]
df2 = df[df['hotel_id']!=1]

Ovo sam eksportovao kasnije mozda za loadovanje, mada nije neophodno

In [30]:
df1.to_csv('../prep_data/prepped1.csv')
df2.to_csv('../prep_data/prepped0.csv')

Ovde sam relativno uspesno pokusao da napravim funkciju koja sracunava broj gostiju za svaki datum.. mada mi se cini da je moglo jednostavnije. Al u sustini ovo uzima intervale ako su izmedju datuma dolaska i rezervacije i dodaje na njih broj_gostiju te konkretne rezervacije.

Pritom je wildly neefikasan algoritam moglo bi zasigurno bolje al ovo je za sad

In [31]:
def calculate_total_occupancy(input_df, frequency='D'):
    df = input_df.copy()

    date_range = pd.date_range(df['datum_dolaska'].min(), df['datum_odjave'].max(), freq=frequency)

    occupancy_df = pd.DataFrame(index=date_range, columns=['total_occupancy'])

    for _, reservation in df.iterrows():
        occupied_days = pd.date_range(reservation['datum_dolaska'], reservation['datum_odjave'], freq=frequency)
        occupancy_df.loc[occupied_days, 'total_occupancy'] = occupancy_df.loc[occupied_days, 'total_occupancy'].add(reservation['broj_gostiju'], fill_value=0)

    # merdzujemo datume dolaska kako bi se mapirale vrednosti korektno
    df = pd.merge(df, occupancy_df, left_on='datum_dolaska', right_index=True, how='left')

    df.drop_duplicates(subset='datum_dolaska', keep='first', inplace=True)
    df = df[df['total_occupancy'].notnull()]
    return df[['datum_dolaska','total_occupancy']]

In [32]:
result_df1 = calculate_total_occupancy(df1)
print(result_df1)

      datum_dolaska total_occupancy
28927    2015-11-01           193.0
28928    2015-07-02            58.0
28929    2015-10-03           235.0
28930    2015-07-03            67.0
28931    2015-12-03           223.0
...             ...             ...
68107    2017-10-24           349.0
68229    2017-10-25           381.0
68278    2017-05-25           238.0
68333    2017-05-27           256.0
68427    2017-05-28           261.0

[940 rows x 2 columns]


In [33]:
result_df2 = calculate_total_occupancy(df2)
print(result_df2)

      datum_dolaska total_occupancy
0        2015-07-01            72.0
28       2015-07-02           140.0
54       2015-07-03           190.0
77       2015-07-04           248.0
100      2015-07-05           310.0
...             ...             ...
27580    2017-07-27           495.0
27679    2017-07-30           501.0
27737    2017-07-31           497.0
27902    2017-08-05           530.0
28717    2017-08-30           460.0

[793 rows x 2 columns]


Postavljam indekse radi eksportovanja

In [34]:
result_df1 = result_df1.set_index('datum_dolaska')
result_df2 = result_df2.set_index('datum_dolaska')

Napravite folder prep_data pre narednog cell-a

In [35]:
result_df2.to_csv('../prep_data/hotelid0_daily_occupancy.csv')
result_df1.to_csv('../prep_data/hotelid1_daily_occupancy.csv')

Sad treba isto uraditi za nedeljno i mesecno, naredni cell ce se dugo izvrsavati jer python a i kod mi nije najefikasniji i dalje, ali samo strpljivo

In [36]:
result_weekly_df1 = calculate_total_occupancy(df1, frequency='W-Mon') # W-Mon je za nedelje i da pocnu u ponedeljak
result_monthly_df1 = calculate_total_occupancy(df1, frequency='M')

result_weekly_df2 = calculate_total_occupancy(df2, frequency='W-Mon')
result_monthly_df2 = calculate_total_occupancy(df2, frequency='M')

In [37]:
result_weekly_df1  = result_weekly_df1.set_index('datum_dolaska')
result_monthly_df1 = result_monthly_df1.set_index('datum_dolaska')

result_weekly_df2  = result_weekly_df2.set_index('datum_dolaska')
result_monthly_df2 = result_monthly_df2.set_index('datum_dolaska')

In [38]:
result_weekly_df1.head()

,total_occupancy
datum_dolaska,
2015-05-04,82.0
2015-05-11,165.0
2015-05-18,167.0
2015-07-20,56.0
2015-11-23,236.0


In [39]:
result_monthly_df1.head()

,total_occupancy
datum_dolaska,
2015-11-30,241.0
2015-07-31,34.0
2015-05-31,95.0
2015-08-31,116.0
2015-01-31,9.0


In [41]:
df0_occupancy = pd.read_csv("../prep_data/hotelid0_daily_occupancy.csv")
df0_dataset = pd.read_csv("../prep_data/prepped0.csv")

df0_occupancy['datum_dolaska'] = pd.to_datetime(df0_occupancy['datum_dolaska'])
df0_dataset['datum_dolaska'] = pd.to_datetime(df0_dataset['datum_dolaska'])

df0 = pd.merge(df0_dataset, df0_occupancy, on='datum_dolaska', how='left')
df0 = df0.drop(columns = ["Unnamed: 0", "broj_gostiju","index","hotel_id"])
df0.to_csv('../prep_data/hotelid1_daily_merged.csv')

In [42]:
df1_occupancy = pd.read_csv("../prep_data/hotelid1_daily_occupancy.csv")
df1_dataset = pd.read_csv("../prep_data/prepped1.csv")

df1_occupancy['datum_dolaska'] = pd.to_datetime(df1_occupancy['datum_dolaska'])
df1_dataset['datum_dolaska'] = pd.to_datetime(df1_dataset['datum_dolaska'])

df1 = pd.merge(df1_dataset, df1_occupancy, on='datum_dolaska', how='left')
df1 = df1.drop(columns = ["Unnamed: 0", "broj_gostiju","index","hotel_id"])
df1.to_csv('../prep_data/hotelid1_daily_merged.csv')